Scrapping using beautifulsoup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import requests
from bs4 import BeautifulSoup
import csv

# Kata kunci pencarian
kata_kunci = input("Masukkan kata kunci: ")
kata_kunci = kata_kunci.replace(' ', '+')

sort_input = int(input('''pilih urutan berdasarkan
ketik 1 untuk artikel yang relevan
ketik 0 untuk artikel yang terbaru :
'''))

# Atur parameter urutan berdasarkan input pengguna
sort = 'Relevance' if sort_input == 1 else 'PubDate'

# Jumlah baris maksimum
max_rows = int(input("Masukkan jumlah artikel yang dicari: "))

# Membuka file CSV
with open('/content/drive/MyDrive/pemrosesan text/hasil_scraping.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Menulis header
    writer.writerow(["No", "Judul Artikel", "Nama Jurnal", "Prosiding", "Penulis", "Tahun Terbit", "URL Artikel", "URL PDF", "Abstrak"])

    # Inisialisasi nomor baris
    row_number = 0

    # Looping halaman
    for page in range(1, max_rows):
        # Jika jumlah baris maksimum telah tercapai, hentikan loop
        if row_number >= max_rows:
            break

        # URL SpringerOpen
        url = f"https://www.springeropen.com/search?searchType=publisherSearch&sort={sort}&query={kata_kunci}&page={page}"

        # Membuat permintaan ke SpringerOpen
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Mencari semua elemen dengan class 'c-list-group__item'
        results = soup.find_all('li', {'class': 'c-listing__item u-keyline'})

        # Looping melalui setiap hasil dan mengekstrak informasi
        for result in results:
            # Tambahkan nomor baris
            row_number += 1

            # Jika jumlah baris maksimum telah tercapai, hentikan loop
            if row_number > max_rows:
                break

            title = result.find('h3', {'class': 'c-listing__title'}).text if result.find('h3', {'class': 'c-listing__title'}) else "N/A"
            journal_info = result.find('em',{'data-test': 'journal-title'}).text if result.find('em',{'data-test': 'journal-title'}) else "N/A"
            url_article_element = result.find('a')
            url_article = "https:" + url_article_element['href'] if url_article_element and url_article_element.has_attr('href') else "N/A"

            author = result.find('span', {'class': 'c-listing__authors-list'}).text if result.find('span', {'class': 'c-listing__authors-list'}) else "N/A"
            year_published = result.find('span', {'itemprop': 'datePublished'}).text if result.find('span', {'itemprop': 'datePublished'}) else "N/A"

            pdf_link_element = result.find('a', {'data-test': 'pdf-link'})
            pdf_link = "https:" + pdf_link_element['href'] if pdf_link_element and pdf_link_element.has_attr('href') else "N/A"

            # Membuat permintaan ke halaman artikel
            url_abstract_element = result.find('a', itemprop='url')
            abstract_link = "https:" + url_abstract_element['href']
            article_response = requests.get(abstract_link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Temukan div dengan class 'c-article-section__content'
            result_abstract = article_soup.find('div', {'class': 'c-article-section__content'})

            # Ekstrak teks dalam tag <p> di dalam div
            abstract = result_abstract.find('p').text if result_abstract and result_abstract.find('p') else "N/A"

            # Mengubah abstrak menjadi teks biasa (ASCII)
            abstract_plain = abstract.encode('ascii', 'ignore').decode('ascii')

            # Menghapus spasi ekstra
            abstract_plain = " ".join(abstract_plain.split())

            # Menulis baris ke file CSV
            writer.writerow([row_number, title, journal_info, "N/A", author, year_published, url_article, pdf_link, abstract_plain])


Masukkan kata kunci: speech to text 
pilih urutan berdasarkan
ketik 1 untuk artikel yang relevan
ketik 0 untuk artikel yang terbaru :
1
Masukkan jumlah artikel yang dicari: 10


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import re
import json

# URL GSM Arena
url = "https://www.gsmarena.com/makers.php3"

# Define a function to convert currency to IDR
def convert_to_idr(amount, from_currency):
    api_url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response = requests.get(api_url)
    data = json.loads(response.text)
    return amount * data['rates']['IDR']


# Membuat permintaan ke GSM Arena
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Mencari semua elemen dengan class 'st-text'
results = soup.find_all('div', {'class': 'st-text'})

# Mengambil href dari setiap tag a dalam hasil pencarian
base_url = "https://www.gsmarena.com/"
for result in results:
    links = result.find_all('a', href=True)
    for link in links:
        time.sleep(random.uniform(1, 3))  # Add random delay between 1 to 3 seconds
        href = link['href']
        parts = href.split('-')
        maker, model, id = parts[0], parts[1], parts[2].split('.')[0]
        page_number = 1  # Start from page 1
        while True:  # Keep looping until there's no more pages
            new_href = f"{maker}-{model}-f-{id}-0-p{page_number}.php"
            href_merek = base_url + new_href
            merek_response = requests.get(href_merek)
            merek_soup = BeautifulSoup(merek_response.text, 'html.parser')
            link_merek = merek_soup.find_all('div', {'class': 'makers'}) 
            if not link_merek:  # If there's no more pages, break the loop
                break
            for result_merek in link_merek:
                links_tipe = result_merek.find_all('a', href=True)
                for link_tipe in links_tipe:
                    time.sleep(random.randint(1, 3))  # Add delay between 1 to 3 seconds
                    href_tipe = base_url + link_tipe['href']
                    tipe_response = requests.get(href_tipe)
                    tipe_soup = BeautifulSoup(tipe_response.text, 'html.parser')
                    #Extract the release year
                    release_year_tag = tipe_soup.find('td', {'data-spec': 'year'})
                    if release_year_tag:
                        release_year = int(release_year_tag.text.split(', ')[1])  # Assuming the text is in the format 'Announced, 2020'
                        if release_year < 2020:
                            continue  # Skip this phone if it was released before 2020
                        # Extract the price
                        price_tag = tipe_soup.find('td', {'data-spec': 'price'}).text or tipe_soup.find_all('a', href=True).text
                        if len(price_tag) <20:
                            href_harga = links_tipe['href']
                            parts = href.split('-')
                            id_href_harga = parts[2].split('.')[0]
                            new_format = f"price.php3?idPhone={id_href_harga.group}&sSaveCurrency=IDR"
                            href_harga = base_url + new_format
                            harga_response = requests.get(href_harga)
                            harga_soup = BeautifulSoup(harga_response.text, 'html.parser')
                            harga_tag = harga_soup.find('a', {'data-url': re.compile(r'^price-js\.php3')})
                            print(harga_tag)
                        else:
                            # Extract the numeric part of the price and the currency code
                            price_parts = price_tag.split(' ')
                            price_amount = float(price_parts[1].replace(',', ''))
                            price_currency = price_parts[2]
                            # Convert the price to IDR
                            price_in_idr = convert_to_idr(price_amount, price_currency)
                            print(price_in_idr)
            page_number += 1  # Increment the page number for the next iteration
